In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, merge, Dense, TimeDistributed, Dropout
from keras.layers import Bidirectional, concatenate, SpatialDropout1D
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from numpy import dot


import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import add



Using TensorFlow backend.


In [2]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [3]:
data = data.fillna(method="ffill")
data.tail(5)

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [4]:
wordwa = list(set(data["Word"].values))
words = []

for w in wordwa :
    w = str(w)
    w = w.lower()
    words.append(w)
print(words[0:10])
n_words = len(words); 
n_words

['machar', 'spector', 'hamas-dominated', 'terrorist-related', 'mosleshi', 'moiseyev', 'mihtarlam', 'must', 'four-time', 'three']


35178

In [5]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

72


In [6]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [7]:
print(char2idx['k'])
print(char2idx['-'])

34
15


In [8]:
max_word_length = 25
embedded_char_vector_length = 74 # Length of character dictionary
char_feature_output = 10

In [9]:
X_char = []
for word in words :
    l1 = []
    #if (len(word) > max_word_length) :
    #    print(len(word))
    for i in range(max_word_length):
        try:
            l1.append(char2idx.get(word[i]))
        except:
            l1.append(char2idx.get("PAD"))    
    
    X_char.append(l1)

In [10]:
print(X_char[0:5])

[[67, 3, 24, 69, 3, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [13, 55, 5, 24, 58, 68, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [69, 3, 67, 3, 13, 15, 43, 68, 67, 70, 64, 3, 58, 5, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [58, 5, 35, 35, 68, 35, 70, 13, 58, 15, 35, 5, 48, 3, 58, 5, 43, 0, 0, 0, 0, 0, 0, 0, 0], [67, 68, 13, 48, 5, 13, 69, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [11]:
char_input = Input(shape=(max_word_length,), dtype='float32', name='char_input')
char_input1 = Embedding(embedded_char_vector_length, 16, input_length = max_word_length)(char_input)
char_input2 = Dropout(0.2)(char_input1)
'''lstm_out_forward = LSTM(char_feature_output, dropout=0.2, recurrent_dropout=0.2)(char_input2)
lstm_out_backward = LSTM(char_feature_output, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(char_input2)'''

merged = (Bidirectional(LSTM(units=5, return_sequences=False,trainable = False,
                                recurrent_dropout=0.5)))(char_input2)
#merged = merge([lstm_out_forward, lstm_out_backward], mode='concat', concat_axis=1)

#merged = add([lstm_out_forward, lstm_out_backward])
#model = Model(input=[char_input], output=[merged])
model = Model(char_input, merged)

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_input (InputLayer)      (None, 25)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 16)            1184      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 16)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10)                880       
Total params: 2,064
Trainable params: 2,064
Non-trainable params: 0
_________________________________________________________________


In [13]:
p = model.layers[1].get_weights()

print(np.array(p).shape)
print(p)

(1, 74, 16)
[array([[-0.00188003, -0.01638674, -0.01519029, ...,  0.03953842,
         0.04103046, -0.04043776],
       [-0.01378814, -0.01274761,  0.00859804, ..., -0.01827698,
         0.02370476, -0.04771221],
       [ 0.03604699, -0.01655985,  0.02410525, ..., -0.0238024 ,
        -0.01044647,  0.00878341],
       ...,
       [-0.01469404,  0.01237424,  0.04353926, ..., -0.02892401,
         0.03186427,  0.03806816],
       [-0.01772611, -0.01272834, -0.02801614, ..., -0.03083432,
         0.04495514,  0.0139717 ],
       [ 0.02678974,  0.00505128,  0.00439842, ...,  0.0195219 ,
        -0.0411659 , -0.01795516]], dtype=float32)]


In [14]:
X1 = np.array(X_char[0:5])
print(X1)
train_char_vectors = model.predict(X1)
        

[[67  3 24 69  3 35  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [13 55  5 24 58 68 35  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [69  3 67  3 13 15 43 68 67 70 64  3 58  5 43  0  0  0  0  0  0  0  0  0
   0]
 [58  5 35 35 68 35 70 13 58 15 35  5 48  3 58  5 43  0  0  0  0  0  0  0
   0]
 [67 68 13 48  5 13 69 70  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]]


In [15]:
print(train_char_vectors.shape)
print("\n")
print(train_char_vectors)

(5, 10)


[[ 0.02360075 -0.05752382 -0.03724067 -0.00131877  0.04969073  0.01534668
   0.03073062  0.03367455 -0.01656215 -0.02331233]
 [ 0.02339999 -0.05733719 -0.03739831 -0.0014376   0.04968469  0.02541785
   0.00581778 -0.01002046 -0.01530177 -0.01155149]
 [ 0.02012022 -0.05458629 -0.03819003 -0.00158399  0.04833465  0.00127899
   0.00428706  0.01371874 -0.0199625  -0.01723275]
 [ 0.01755146 -0.05269655 -0.03817954 -0.00101338  0.04687458 -0.00983456
  -0.00222437  0.00456038  0.01787201  0.00350338]
 [ 0.02332991 -0.05729531 -0.03743525 -0.0014512   0.04967834  0.02162964
   0.00436482  0.00147249 -0.01314118 -0.0195953 ]]


In [16]:
def pred_new_num_enc(s) :
    
    pred = s

    l = []
    ip = []
    k=0

    for i in range(len(pred)):
        l.append(char2idx.get(pred[i]))
    #print(l)
    #print(char2idx.get("PAD"))        


    length = len(l)
    #print(length)

    if length < max_word_length :
        k = max_word_length - length
        #print(k)

        for i in range (0,k) :
            l.append(char2idx.get("PAD"))    

    else :
        l = l[:max_word_length]
    #ip.append(l)

    l = np.array([l])
    #print(l)
    
    return l

In [20]:
def pred_new_vec(l) :
    train_char_vectors = model.predict(l)

    #print(train_char_vectors)
    
    return train_char_vectors

In [23]:
a1 = pred_new_num_enc("name")
a2 = pred_new_num_enc("nam")

print(pred_new_vec(a1))
print(cosine_similarity(pred_new_vec(a1) , pred_new_vec(a2)))

21
22
[[ 0.023634   -0.05762996 -0.03712891 -0.00118482  0.04966611  0.01519991
   0.0433609   0.01719956 -0.01202335 -0.02731748]]
[[0.99834657]]


In [24]:
a3 = pred_new_num_enc("name")
a4 = pred_new_num_enc("nam-e")

print(cosine_similarity(pred_new_vec(a3) , pred_new_vec(a4)))

21
20
[[0.9980316]]


In [25]:
a5 = pred_new_num_enc("address")
a6 = pred_new_num_enc("addr")

print(cosine_similarity(pred_new_vec(a5) , pred_new_vec(a6)))

18
21
[[0.99301356]]


In [26]:
a7 = pred_new_num_enc("address")
a8 = pred_new_num_enc("adres")

print(cosine_similarity(pred_new_vec(a7) , pred_new_vec(a8)))

18
20
[[0.98561466]]


In [27]:
a9 = pred_new_num_enc("contact")
a10 = pred_new_num_enc("phone-no")

print(cosine_similarity(pred_new_vec(a9) , pred_new_vec(a10)))

18
17
[[0.88158864]]


In [28]:
a11 = pred_new_num_enc("sex")
a12 = pred_new_num_enc("gender")

print(cosine_similarity(pred_new_vec(a11) , pred_new_vec(a12)))

22
19
[[0.97072166]]
